In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dacon/submission.csv
/kaggle/input/dacon/test.csv
/kaggle/input/dacon/train.csv


In [2]:
import time
import random
import datetime
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.optimizers import Nadam

# LOAD LIBRARIES
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

# How to check if Keras is using GPU?


train = pd.read_csv('../input/dacon/train.csv')
test  = pd.read_csv('../input/dacon/test.csv')
submission = pd.read_csv('../input/dacon/submission.csv')

In [3]:
X_train = (train[[str(i) for i in range(784)]] / 255.).values.reshape(-1,28,28,1)
y_train = to_categorical(train['digit'].values)

In [4]:
X_letter = train.iloc[:,2].values
for idx in range(len(X_letter)):
    X_letter[idx] = ord(X_letter[idx])
X_letterToNum = X_letter

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [6]:

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import (
    Conv2D,Conv1D, BatchNormalization, Dropout, MaxPool2D, Activation,
    Flatten, Dense, Input, Concatenate, LeakyReLU, Add, AveragePooling2D,GlobalAveragePooling2D
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from keras import regularizers
import numpy as np
import argparse
import locale
import os

# 이진화 시킨 letter data 를 Input 으로 받을 MLP 구성

In [7]:
inputs1 = Input((7,1))

mlp = Dense(16, activation='relu', kernel_initializer='he_normal' ,bias_regularizer=regularizers.l2(l=0.1))(inputs1)
mlp = Dense(8, activation='relu', kernel_initializer='he_normal',bias_regularizer=regularizers.l2(l=0.1))(mlp)
mlp = Dense(4, activation='relu', kernel_initializer='he_normal',bias_regularizer=regularizers.l2(l=0.1))(mlp)
mlp = Flatten()(mlp)

mlp = Model(inputs1, mlp)

# img data 를 Input 으로 받을 ResNet 구성

In [8]:
from keras import layers

In [9]:
 # Remove the previous model.
model = None
    
# Input layer
img_input = layers.Input(shape = (28,28,1))

# CNN
# Identity mapping shortcut을 위한 conv_1 layer
conv_1 = layers.Conv2D(128, kernel_size = 3, padding = 'same', kernel_initializer='he_normal',activation = 'relu')(img_input) 

conv_2_1 = layers.Conv2D(128, kernel_size = 3, padding = 'same', kernel_initializer='he_normal',activation = 'relu')(conv_1)
conv_2_1 = layers.Conv2D(128, kernel_size = 3, padding = 'same', kernel_initializer='he_normal')(conv_2_1)


# ShortCut connection
add_2_1 = layers.add([conv_1, conv_2_1])
add_2_1 = BatchNormalization()(add_2_1)
out_2_1 = layers.Activation('relu')(add_2_1)

conv_2_2 = layers.Conv2D(128, kernel_size = 3, padding = 'same',kernel_initializer='he_normal', activation = 'relu')(out_2_1)
conv_2_2 = layers.Conv2D(128, kernel_size = 3, padding = 'same',kernel_initializer='he_normal')(conv_2_2)

# ShortCut connection
add_2_2 = layers.add([out_2_1, conv_2_2])
add_2_2= BatchNormalization()(add_2_2)
out_2_2 = layers.Activation('relu')(add_2_1)

pool_2 = layers.MaxPool2D((2,2), strides = 2)(out_2_2)

conv_3_0 = layers.Conv2D(256, kernel_size = 1, strides = 1, kernel_initializer='he_normal')(pool_2)

conv_3_1 = layers.Conv2D(256, kernel_size = 3, padding = 'same', activation = 'relu',kernel_initializer='he_normal')(conv_3_0)
conv_3_1 = layers.Conv2D(256, kernel_size = 3, padding = 'same',kernel_initializer='he_normal')(conv_3_1)

# ShortCut connection
add_3_1 = layers.add([conv_3_0, conv_3_1])
add_3_1= BatchNormalization()(add_3_1)

out_3_1 = layers.Activation('relu')(add_3_1)

conv_3_2 = layers.Conv2D(256, kernel_size = 3, padding = 'same', activation = 'relu',kernel_initializer='he_normal')(out_3_1)
conv_3_2 = layers.Conv2D(256, kernel_size = 3, padding = 'same',kernel_initializer='he_normal')(conv_3_2)

# ShortCut connection
add_3_2 = layers.add([out_3_1, conv_3_2])
add_3_2= BatchNormalization()(add_3_2)

out_3_2 = layers.Activation('relu')(add_3_2)

pool_3 = layers.MaxPool2D((2,2), strides = 2)(out_3_2)

conv_4_0 = layers.Conv2D(256, kernel_size = 1, strides = 1)(pool_3)

conv_4_1 = layers.Conv2D(256, kernel_size = 3, padding = 'same', activation = 'relu',kernel_initializer='he_normal')(conv_4_0)
conv_4_1 = layers.Conv2D(256, kernel_size = 3, padding = 'same',kernel_initializer='he_normal')(conv_4_1)




# ShortCut connection
add_4_1 = layers.add([conv_4_0, conv_4_1])
add_4_1= BatchNormalization()(add_4_1)

out_4_1 = layers.Activation('relu')(add_4_1)

pool_4 = layers.MaxPool2D((2,2), strides = 2)(out_4_1)

conv_5_0 = layers.Conv2D(256, kernel_size = 1, strides = 1)(pool_4)

conv_4_2 = layers.Conv2D(256, kernel_size = 3, padding = 'same', activation = 'relu',kernel_initializer='he_normal')(conv_5_0)
conv_4_2 = layers.Conv2D(256, kernel_size = 3, padding = 'same',kernel_initializer='he_normal')(conv_4_1)

# ShortCut connection
add_4_2 = layers.add([out_4_1, conv_4_2])
add_4_2= BatchNormalization()(add_4_2)

out_4_2 = layers.Activation('relu')(add_4_2)

pool_5 = layers.MaxPool2D((2,2), strides = 2)(out_4_2)

conv_5_0 = layers.Conv2D(256, kernel_size = 1, strides = 1)(pool_4)

conv_5_1 = layers.Conv2D(256, kernel_size = 3, padding = 'same', activation = 'relu',kernel_initializer='he_normal')(conv_5_0)
conv_5_1 = layers.Conv2D(256, kernel_size = 3, padding = 'same',kernel_initializer='he_normal')(conv_5_1)





# ShortCut connection
add_5_1 = layers.add([conv_5_0, conv_5_1])
add_5_1= BatchNormalization()(add_5_1)

out_5_1 = layers.Activation('relu')(add_5_1)

pool_5 = layers.MaxPool2D((2,2), strides = 2)(out_5_1)


# FC layers
img_features = layers.Flatten()(pool_5)
img_features = layers.Dense(512, activation = 'relu',kernel_regularizer=regularizers.l2(l=0.1))(img_features)
img_features = layers.Dropout(rate = 0.4)(img_features)
img_features = layers.Dense(512, activation = 'relu',kernel_regularizer=regularizers.l2(l=0.1))(img_features)   
img_features = layers.Dropout(rate = 0.4)(img_features)
    
x = Model(img_input, img_features)

## MLP 와 CNN 합치기

In [10]:
combinedInput = concatenate([mlp.output, x.output])

In [11]:
from tensorflow.keras import regularizers

In [12]:
y = Dense(10, activation="softmax")(combinedInput)


In [13]:
final_model = Model(inputs=[mlp.input, x.input], outputs=y)

In [14]:
final_model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [15]:
X_letterToNum = np.asarray(X_letterToNum).astype(np.float32)
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

## data 늘리기

In [16]:
datagen = ImageDataGenerator(width_shift_range=5,
                                 height_shift_range=5,
                                 rotation_range=90,
                                 zoom_range=0.05)  


flow1 = datagen.flow(X_train, X_letterToNum, batch_size=32, seed=1218)
flow2 = datagen.flow(X_train, y_train, batch_size=32, seed=1218)

In [ ]:
i = 0
while i < 1500:
    X_image_gen1,X_letter_gen=flow1.next()
    X_image_gen2,y_gen=flow2.next()

    X_image_gen1 = np.asarray(X_image_gen1).astype(np.float32)
    X_letter_gen = np.asarray(X_letter_gen).astype(np.float32)
    y_gen = np.asarray(y_gen).astype(np.float32)
    
    X_train = np.r_[X_train, X_image_gen1]
    X_letterToNum = np.r_[X_letterToNum, X_letter_gen]
    y_train = np.r_[y_train,y_gen]
    
    i += 1
    """    
    print("X_train.shape={}".format(X_train.shape))
    print("X_letterToNum.shape={}".format(X_letterToNum.shape))
    print("y_train.shape={}".format(y_train.shape))
    print('\n')
    """


## letter data 이진화

In [ ]:
X_letterToNum = X_letterToNum.astype(np.int)

In [ ]:
X_letterToBi = []
for i in range(len(X_letterToNum)):
    tmp = X_letterToNum[i]
    bi = np.zeros(7)
    k = 6
    while tmp > 0 and k >= 0:
        if tmp >= pow(2, k):
            bi[6-k] = 1
            tmp -= pow(2, k)
        k -= 1
    X_letterToBi.append(bi)

In [ ]:
X_letterToBi = np.asarray(X_letterToBi).astype(np.float32)


## Learning Rate Scheduler 설정

In [ ]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

In [ ]:
epochs = 20

# Multiple Inputs model Fitting

In [ ]:
history = final_model.fit(
    x=[X_letterToBi, X_train], y=y_train,
    validation_split=0.1,
    epochs=epochs, 
    callbacks=[annealer],
    steps_per_epoch=X_train
    .shape[0]//32,
    shuffle=True
)

In [ ]:
plt.figure()

epoch = np.arange(epochs)

plt.plot(epoch, np.asarray(history.history['accuracy']),label="acc")
plt.plot(epoch, np.asarray(history.history['val_accuracy']),label="val_acc")

plt.title('Accuracy vs val_accuracy')
plt.ylim(0,1)

plt.legend(bbox_to_anchor=(1.02,1),loc=2)

# ResNet Model 2 구성

In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models

from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_df = pd.read_csv('../input/dacon/train.csv', usecols = lambda column : column not in ["id" , "letter"])
test_df = pd.read_csv('../input/dacon/test.csv',usecols = lambda column : column not in ["id" , "letter"])
submission = pd.read_csv('../input/dacon/submission.csv')

In [ ]:
# have a glimpse of train dataframe structure
n_train = len(train_df)
n_pixels = len(train_df.columns) - 1
n_class = len(set(train_df['digit']))
print('Number of training samples: {0}'.format(n_train))
print('Number of training pixels: {0}'.format(n_pixels))
print('Number of classes: {0}'.format(n_class))
print(train_df.head())

# have a glimpse of test dataframe structure
n_test = len(test_df)
n_pixels = len(test_df.columns)
print('Number of test samples: {0}'.format(n_test))
print('Number of test pixels: {0}'.format(n_pixels))
print(test_df.head())

In [ ]:
random_sel = np.random.randint(n_train, size=8)

grid = make_grid(torch.Tensor((train_df.iloc[random_sel, 1:].values/255.).reshape((-1, 28, 28))).unsqueeze(1), nrow=8)
plt.rcParams['figure.figsize'] = (16, 2)
plt.imshow(grid.numpy().transpose((1,2,0)))
plt.axis('off')
print(*list(train_df.iloc[random_sel, 0].values), sep = ', ')

In [ ]:
class MNISTDataset(Dataset):
    """MNIST dtaa set"""
    
    def __init__(self, dataframe, 
                 transform = transforms.Compose([transforms.ToPILImage(),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize(mean=(0.5,), std=(0.5,))])
                ):
        df = dataframe
        # for MNIST dataset n_pixels should be 784
        self.n_pixels = 784
        
        if len(df.columns) == self.n_pixels:
            # test data
            self.X = df.values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.y = None
        else:
            # training data
            self.X = df.iloc[:,1:].values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.y = torch.from_numpy(df.iloc[:,0].values)
            
        self.transform = transform
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.transform(self.X[idx]), self.y[idx]
        else:
            return self.transform(self.X[idx])

In [ ]:
RandAffine = transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))

In [ ]:
rotate = transforms.RandomRotation(degrees=0)
shift = RandAffine
composed = transforms.Compose([rotate,
                               shift])

# Apply each of the above transforms on sample.
fig = plt.figure()
sample = transforms.ToPILImage()(train_df.iloc[65,1:].values.reshape((28,28)).astype(np.uint8)[:,:,None])
for i, tsfrm in enumerate([rotate, shift, composed]):
    transformed_sample = tsfrm(sample)

    ax = plt.subplot(1, 3, i + 1)
    plt.tight_layout()
    ax.set_title(type(tsfrm).__name__)
    ax.imshow(np.reshape(np.array(list(transformed_sample.getdata())), (-1,28)), cmap='gray')    

plt.show()

In [ ]:
batch_size = 64

train_transforms = transforms.Compose(
    [transforms.ToPILImage(),
     RandAffine,
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.5,), std=(0.5,))])

val_test_transforms = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.5,), std=(0.5,))])

def get_dataset(dataframe, dataset=MNISTDataset,
                transform=transforms.Compose([transforms.ToPILImage(),
                                              transforms.ToTensor(),
                                              transforms.Normalize(mean=(0.5,), std=(0.5,))])):
    return dataset(dataframe, transform=transform)

In [ ]:
resnet18 = models.resnet18()

In [ ]:
from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck

class MNISTResNet(ResNet):
    def __init__(self):
        super(MNISTResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10) # Based on ResNet18
        #super(MNISTResNet, self).__init__(BasicBlock, [3, 4, 6, 3], num_classes=10) # Based on ResNet34
        #super(MNISTResNet, self).__init__(Bottleneck, [3, 4, 6, 3], num_classes=10) # Based on ResNet50
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=3,bias=False)

model = MNISTResNet()
print(model)

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch):
    model.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader): 
        # if GPU available, move data and target to GPU
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        # compute output and loss
        output = model(data)
        loss = criterion(output, target)
        
        # TODO:
        # 1. add batch metric (acc1, acc5)
        # 2. add average metric top1=sum(acc1)/batch_idx, top5 = sum(acc5)/batch_idx
        
        # backward and update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    print('Train Epoch: {} , Accuracy: {}/{} ({:.3f}%)'.format(
        epoch ,correct, len(train_loader.dataset),100.0 * float(correct) / len(train_loader.dataset)))

In [ ]:
def validate(val_loader, model, criterion):
    model.eval()
    loss = 0
    correct = 0
    
    for _, (data, target) in enumerate(val_loader):
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        
        loss += criterion(output, target).data.item()

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(val_loader.dataset)
        
    print('On Val set Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)'.format(
        loss, correct, len(val_loader.dataset),
        100.0 * float(correct) / len(val_loader.dataset)))

In [ ]:
total_epoches = 1
step_size = 1
base_lr = 1

optimizer = optim.Adam(model.parameters(), lr=base_lr)
criterion = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
def split_dataframe(dataframe=None, fraction=0.9, rand_seed=1):
    df_1 = dataframe.sample(frac=fraction, random_state=rand_seed)
    df_2 = dataframe.drop(df_1.index)
    return df_1, df_2
for epoch in range(total_epoches):
    print("\nTrain Epoch {}: lr = {}".format(epoch, exp_lr_scheduler.get_lr()[0]))

    train_df_new, val_df = split_dataframe(dataframe=train_df, fraction=0.9, rand_seed=epoch)
    
    train_dataset = get_dataset(train_df_new, transform=train_transforms)
    val_dataset = get_dataset(val_df, transform=val_test_transforms)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                             batch_size=batch_size, shuffle=True)

In [ ]:
total_epoches = 30
step_size = 10
base_lr = 0.01

optimizer = optim.Adam(model.parameters(), lr=base_lr)
criterion = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)

for epoch in range(total_epoches):
    print("\nTrain Epoch {}: lr = {}".format(epoch, exp_lr_scheduler.get_lr()[0]))

    train(train_loader=train_loader, model=model, criterion=criterion, optimizer=optimizer, epoch=epoch)
    validate(val_loader=val_loader, model=model, criterion=criterion)
    exp_lr_scheduler.step()

In [ ]:
total_epoches = 30
step_size = 10
base_lr = 0.01

optimizer = optim.Adam(model.parameters(), lr=base_lr)
criterion = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)

for epoch in range(total_epoches):
    print("\nTrain Epoch {}: lr = {}".format(epoch, exp_lr_scheduler.get_lr()[0]))

    train_df_new, val_df = split_dataframe(dataframe=train_df, fraction=0.9, rand_seed=epoch)
    
    train_dataset = get_dataset(train_df_new, transform=train_transforms)
    val_dataset = get_dataset(val_df, transform=val_test_transforms)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                             batch_size=batch_size, shuffle=True)

    train(train_loader=train_loader, model=model, criterion=criterion, optimizer=optimizer, epoch=epoch)
    validate(val_loader=val_loader, model=model, criterion=criterion)
    exp_lr_scheduler.step()

In [ ]:
def prediciton(test_loader, model):
    model.eval()
    test_pred = torch.LongTensor()
    
    for i, data in enumerate(test_loader):
        if torch.cuda.is_available():
            data = data.cuda()
            
        output = model(data)
        
        pred = output.cpu().data
        if i == 0 :
            test_pred = pred
        else :
            test_pred = torch.cat([test_pred,pred], dim=0)
        """
        pred = output.cpu().data.max(1, keepdim=True)[1]
        test_pred = torch.cat((test_pred, pred), dim=0)
        """
        del pred
        
    return test_pred

# Ensemble of Model1 and Model2

In [ ]:
test_batch_size = 64
test_dataset = get_dataset(test_df, transform=val_test_transforms)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=test_batch_size, shuffle=False)

# tensor prediction
pred_output = prediciton(test_loader, model)

In [ ]:
X_test_letter = test.iloc[:,1].values

In [ ]:
for idx in range(len(X_test_letter)):
    X_test_letter[idx] = ord(X_test_letter[idx])
X_letterToNum_test = X_test_letter.astype(np.int)

In [ ]:
X_letterToBi_test = []
for i in range(len(X_letterToNum_test)):
    tmp = X_letterToNum_test[i]
    bi = np.zeros(7)
    k = 6
    while tmp > 0 and k >= 0:
        if tmp >= pow(2, k):
            bi[6-k] = 1
            tmp -= pow(2, k)
        k -= 1
    X_letterToBi_test.append(bi)

In [ ]:
X_letterToBi_test = np.asarray(X_letterToBi_test).astype(np.float32)

In [ ]:
X_test = (test[[str(i) for i in range(784)]] / 255.).values.reshape(-1,28,28,1)

pred_output_keras = final_model.predict([X_letterToBi_test,X_test])


In [ ]:
results = np.argmax((pred_output + pred_output_keras) / 2,axis=1)

In [ ]:
submission.digit = results
submission.to_csv('submission_09-14_앙상블_마지막.csv', index=False) # 0.795575446%